In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [4]:
corpus= ["apple ball cat", "ball cat dog elephant"]

In [5]:
max_features=4
n_grams= 2

In [6]:
#Convert a collection of text to a matrix of token counts
vectorizer= CountVectorizer(max_features= max_features, ngram_range=(1,n_grams))
X= vectorizer.fit_transform(corpus)


In [5]:
print(X.toarray())

[[1 1 1 1]
 [0 1 1 1]]


In [6]:
print(vectorizer.get_feature_names_out())

['apple' 'ball' 'ball cat' 'cat']


In [7]:
#Testing feature extraction
import os
import numpy as np
from src.utils.common import get_df
import pandas as pd

os.chdir('../')

In [27]:

df_train= pd.read_csv('/Users/nithinpradeep/Projects/NLP-classification-with-DVC/artifacts/prepared/train.tsv',
                delimiter="\t",
                encoding='utf-8',
                #header=None,
                #names=["id","label","text"]
                )
df_train.head(3)

,pid,label,text
0,4186249,1,Searching and capturing a character using regu...
1,54312,0,"<p>Yes, of course there is :-) object oriente..."
2,7956164,0,<p>Try waiting for window's <code>load</code>...


In [29]:
train_words= np.array(df_train.text.str.lower().values.astype("U"))#convert to unicode
train_words[0]
len(df_train)

17536

In [30]:

bag_of_words = CountVectorizer(
        stop_words="english",
        max_features=max_features,
        ngram_range=(1, n_grams)
    )
bag_of_words.fit(train_words)
train_words_binary_matrix = bag_of_words.transform(train_words)
train_words_binary_matrix

<17536x4 sparse matrix of type '<class 'numpy.int64'>'
	with 33149 stored elements in Compressed Sparse Row format>

In [31]:
tfidf = TfidfTransformer(smooth_idf=False)
tfidf.fit(train_words_binary_matrix)
train_words_tfidf_matrix = tfidf.transform(train_words_binary_matrix)
train_words_tfidf_matrix

<17536x4 sparse matrix of type '<class 'numpy.float64'>'
	with 33149 stored elements in Compressed Sparse Row format>

In [1]:
#Save matrix operation
import scipy.sparse as sparse

In [32]:
df_train= pd.read_csv('/Users/nithinpradeep/Projects/NLP-classification-with-DVC/artifacts/prepared/train.tsv',
                delimiter="\t",
                encoding='utf-8',
                #header=None,
                #names=column_names
                )

In [37]:
pid_matrix= sparse.csr_matrix(df_train.pid.astype(np.int64)).T 
label_matrix= sparse.csr_matrix(df_train.label.astype(np.int64)).T 

result= sparse.hstack([pid_matrix,label_matrix,train_words_tfidf_matrix], format="csr")
result

<17536x6 sparse matrix of type '<class 'numpy.float64'>'
	with 51031 stored elements in Compressed Sparse Row format>

In [35]:
pid_matrix.shape, label_matrix.shape, train_words_tfidf_matrix.shape, result.shape

((17536, 1), (17536, 1), (17536, 4), (17536, 6))